In [320]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd
import difflib

dataDir = Path('../dataset/images') 
annotationsDir = Path('../dataset/annotations')

In [321]:
# Image Pre-processing

# Improve Lighting

def improve_lighting(img):
    imgYUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    imgYUV[:, :, 0] = cv2.equalizeHist(imgYUV[:, :, 0])

    imgBetterLighting = cv2.cvtColor(imgYUV, cv2.COLOR_YUV2BGR)

    imgHSV = cv2.cvtColor(imgBetterLighting, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(imgHSV)

    lim = 255 - 50
    v[v > lim] = 255
    v[v <= lim] += 50

    final_hsv = cv2.merge((h, s, v))
    imgBetterLighting = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

    return imgBetterLighting

# Smooth

def image_smooth(img):
    imgWithMedianFilter = cv2.medianBlur(img, 5)

    return imgWithMedianFilter


In [322]:
# Image Segmentation

def image_segmentation(img):
    #set the bounds for the red hue
    lower_red_n1 = np.array([0,70,60])
    upper_red_n1 = np.array([10,255,255])

    lower_red_n2 = np.array([170,70,60])
    upper_red_n2 = np.array([180,255,255])

    lower_blue_n3 = np.array([78,158,124])
    upper_blue_n3 = np.array([138,255,255])

    #create a mask using the bounds set

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)


    mask_1 = cv2.inRange(img_hsv, lower_red_n1, upper_red_n1)
    mask_2 = cv2.inRange(img_hsv, lower_red_n2, upper_red_n2)

    mask_red = mask_1 + mask_2

    mask_blue = cv2.inRange(img_hsv, lower_blue_n3, upper_blue_n3)

    return mask_blue, mask_red

In [323]:
# Image Thresholding and Morphological Operations

def morphological_ops(img):
    # Red

    # Removing Noise
    kernel = np.ones((3, 3),np.uint8)
    processed_red = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations = 1)
    processed_red = cv2.morphologyEx(processed_red, cv2.MORPH_DILATE, kernel, iterations = 1)

    # Floodfill

    red_floodfill = processed_red.copy()

    h, w = processed_red.shape[:2]

    mask = np.zeros((h+2, w+2), np.uint8)

    cv2.floodFill(red_floodfill, mask, (0,0), 255)

    red_floodfill_inv = cv2.bitwise_not(red_floodfill)

    filled_image = processed_red | red_floodfill_inv

    return filled_image

In [324]:
# Shape Recognition

def shape_recognition(img_red, img_blue, initial_image):
    
    result = ""
    results = []
    img_red_contours = initial_image
    img_blue_contours = initial_image

    # Red
    debug = "red"

    # Octagon Detection
    contours_red, hierarchy_red = cv2.findContours(img_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours_red:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        #print(len(approx))

        if len(approx) == 8:
            # print('Found STOP sign')
            result = "stop"
            results.append("stop")
            img_red_contours = cv2.drawContours(initial_image, [cnt], 0, (0,0,255), -1)

        elif len(approx) >= 12:
            # print('Found red circle sign')
            result = "speedlimit"
            results.append("speedlimit")
            img_red_contours = cv2.drawContours(initial_image, [cnt], 0, (0,0,255), -1)

    # Blue

    contours_blue, hierarchy_blue = cv2.findContours(img_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours_blue:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        # print(len(approx))

        if len(approx) == 4:
            # print('Found BLUE sign')
            result = "crosswalk"
            results.append("crosswalk")
            # debug = "blue"
            img_blue_contours = cv2.drawContours(initial_image, [cnt], 0, (255,0,0), -1)

    """
    if debug=="red":
        plt.imshow(cv2.cvtColor(img_red_contours, cv2.COLOR_BGR2RGB))
        plt.title('Foreground')
        plt.axis('off')
        plt.show()
    elif debug == "blue":
        plt.imshow(cv2.cvtColor(img_blue_contours, cv2.COLOR_BGR2RGB))
        plt.title('Foreground')
        plt.axis('off')
        plt.show() 
    """
    
    return result, results


In [325]:
def evaluate_image(imgPath):
    
    img = cv2.imread(imgPath)

    imgLighting = improve_lighting(img)
    imgSmooth = image_smooth(imgLighting)
    img_blue, img_red = image_segmentation(imgSmooth)
    processed_blue = morphological_ops(img_blue)
    processed_red = morphological_ops(img_red)
    result = shape_recognition(processed_red, processed_blue, img)
    
    return result

In [326]:
# Classification setup

def filelist(root, file_type):
    return [os.path.join(directory_path, f) for directory_path, directory_name, 
            files in os.walk(root) for f in files if f.endswith(file_type)]

def generate_train_df (anno_path):
    annotations = filelist(anno_path, '.xml')
    anno_list = []
    for anno_path in annotations:
        root = ET.parse(anno_path).getroot()
        anno = {}
        anno['filename'] = str(dataDir) + '/'+ root.find("./filename").text
        #anno['width'] = root.find("./size/width").text
        #anno['height'] = root.find("./size/height").text
        classArray = []
        for child in root:
            if child.tag == "object":
                for grandchild in child:
                    if grandchild.tag == "name":
                        if grandchild.text != "trafficlight":
                            classArray.append(grandchild.text)
        anno['expected'] = classArray
        anno_list.append(anno)
    return pd.DataFrame(anno_list)

df_train = generate_train_df(annotationsDir)
print(df_train.shape)
df_train.head()

(877, 2)


,filename,expected
0,../dataset/images/road712.png,"[speedlimit, speedlimit]"
1,../dataset/images/road706.png,"[speedlimit, speedlimit]"
2,../dataset/images/road289.png,[stop]
3,../dataset/images/road538.png,[speedlimit]
4,../dataset/images/road510.png,[speedlimit]


In [327]:
# Creating prediction Dataframe

def generate_prediction():
    pred_list = []
    for index, row in df_train.iterrows():
        pred = {}
        pred["filename"] = row["filename"]
        result, results = evaluate_image(row["filename"])
        pred["prediction"] = results
        pred_list.append(pred)
    return pd.DataFrame(pred_list)

df_pred = generate_prediction()
print(df_pred.shape)
df_pred.head()
    

(877, 2)


,filename,prediction
0,../dataset/images/road712.png,"[speedlimit, stop, speedlimit, speedlimit]"
1,../dataset/images/road706.png,[]
2,../dataset/images/road289.png,"[speedlimit, crosswalk, crosswalk, crosswalk, ..."
3,../dataset/images/road538.png,[]
4,../dataset/images/road510.png,[speedlimit]


In [328]:
# Merging dataframes

def join_dataframes(df_train, df_pred):
    df_merged = pd.merge(df_train, df_pred, on='filename')

    for index, row in df_merged.iterrows():
        sm = difflib.SequenceMatcher(None,row["expected"],row["prediction"])
        df_merged.at[index,"Similarity"] = sm.ratio()

    return df_merged

df_merged = join_dataframes(df_train, df_pred)
print(df_merged.shape)
df_merged.head()

(877, 4)


,filename,expected,prediction,Similarity
0,../dataset/images/road712.png,"[speedlimit, speedlimit]","[speedlimit, stop, speedlimit, speedlimit]",0.666667
1,../dataset/images/road706.png,"[speedlimit, speedlimit]",[],0.000000
2,../dataset/images/road289.png,[stop],"[speedlimit, crosswalk, crosswalk, crosswalk, ...",0.000000
3,../dataset/images/road538.png,[speedlimit],[],0.000000
4,../dataset/images/road510.png,[speedlimit],[speedlimit],1.000000


In [330]:
# Base Classification

print(df_merged["Similarity"].mean())

0.2746249143566899
0/0
